This notebook requires the example file "Maize.apsimx" from the example folder in APSIM Next Generation. To preserve original trait values, the output is written to "Maize_edited.apsimx" and all out files are under this name (for example, "Maize_edited.db". Readme file contains Python version, APSIMX version, and other necessary software.

# Importing necessary libraries

In [1]:
import os
import sys
import fileinput
import subprocess
import re
import optuna
import matplotlib.pyplot as plt
import json

# Defining the objective function

In this example, Optuna will try different combinations of CultivarName (categorical), Fertilizer amount (integer, kg/ha), and residue amount (integer, kg/ha). APSIM is then run in the shell using this edited file. The output .db file is queried with SQLite to extract the max value for the chosen variable (Maize.Grain.Wt in this case). This is returned as "value", a float variable. If the output variable isn't found, a value of 0 is returned as default.

In [2]:
varieties = ['Hycorn_53','Pioneer_33M54','Pioneer_38H20','Pioneer_34K77','Pioneer_39V43','Atrium','Laila','GH_5019WX',
             'Hycorn_40','GH_5009','Dekalb_XL82','malawi_local','mh19','mh17','mh16','mh12','sc623','sc625','sc601',
             'CG4141','mh18','r215','Melkassa','sr52','sc501','r201','sc401','NSCM_41','Makueni','Katumani','Pioneer_3153',
             'Pioneer_39G12','B_80','B_90','B_95','B_100','B_103','B_105','B_108','B_110','B_112','B_115','B_120','B_115',
             'B_120','B_130','A_80','A_90','A_95','A_100','A_103','A_105','A_108','A_110','A_112','A_115','A_120','A_130',
             'HY_110','LY_110','P1197']

This defines the path to the output database, and the query that defines the output variable that Optuna uses to evaluate each trial.

In [3]:
path_to_output_db = "c:/Users/detemplj/Downloads/Maize_edited.db"
output_variable = "SELECT AVG(\"Maize.Grain.Wt\") FROM Report;"

Objective function:

In [4]:
def objective(trial):
    value = None  # Default value

    #### INPUT VARIABLE(S) IS SPECIFIED HERE:
    CultivarName = trial.suggest_categorical('CultivarName', varieties)
    fertiliser = trial.suggest_int('init_fert', 0, 250)
    residue = trial.suggest_int('residue', 250, 1000)
    
    # Changing file variables
    # Read in the file
    with open('Maize.apsimx', 'r') as file:
        data = json.load(file)
    # Replace the target strings
    
    data['Children'][0]['Children'][4]['Children'][7]['Parameters'][5]['Value'] = CultivarName
    data['Children'][0]['Children'][4]['Children'][6]['Parameters'][0]['Value'] = fertiliser
    data['Children'][0]['Children'][4]['Children'][3]['InitialResidueMass'] = residue
    
    # Write the file out again
    with open("Maize_edited.apsimx", 'w') as file:
        json.dump(data, file)
        
      
    
    # Running APSIM in shell
    list_files = subprocess.check_output(["apsim", "run", "Maize_edited.apsimx"], shell=True)
    list_files = list_files.decode("utf-8")
    print(list_files)
    
    # Getting APSIM output
    #### OUTPUT VARIABLE IS SPECIFIED HERE IN THE SQLITE QUERY:
    output = subprocess.check_output(["sqlite3", path_to_output_db, "-cmd", output_variable], shell=True)
    output = output.decode("utf-8")

    match = re.search(r'\d+\.\d+', output)
    if match:
        value = match.group()
        print("The value is:", value)
    else:
        print("No value found.")

    #return float(value)

    if value is not None:
        return float(value)
    else:
        return float('0')

# Running an example study

In [5]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=300)

study.best_params

[I 2023-06-19 16:37:33,207] A new study created in memory with name: no-name-5599a2e8-f7ba-4ef1-8bd7-b6c000288cf8
[I 2023-06-19 16:37:36,181] Trial 0 finished with value: 569.716566773045 and parameters: {'CultivarName': 'Atrium', 'init_fert': 134, 'residue': 740}. Best is trial 0 with value: 569.716566773045.



The value is: 569.716566773045


[I 2023-06-19 16:37:39,147] Trial 1 finished with value: 632.886655994555 and parameters: {'CultivarName': 'A_110', 'init_fert': 137, 'residue': 911}. Best is trial 1 with value: 632.886655994555.



The value is: 632.886655994555


[I 2023-06-19 16:37:42,113] Trial 2 finished with value: 295.758243946052 and parameters: {'CultivarName': 'Atrium', 'init_fert': 19, 'residue': 326}. Best is trial 1 with value: 632.886655994555.


...

[I 2023-06-19 16:52:42,608] Trial 298 finished with value: 611.41398652995 and parameters: {'CultivarName': 'B_120', 'init_fert': 151, 'residue': 978}. Best is trial 187 with value: 820.706242879662.



The value is: 611.41398652995


[I 2023-06-19 16:52:45,635] Trial 299 finished with value: 643.114421318765 and parameters: {'CultivarName': 'B_115', 'init_fert': 147, 'residue': 994}. Best is trial 187 with value: 820.706242879662.



The value is: 643.114421318765


{'CultivarName': 'HY_110', 'init_fert': 148, 'residue': 664}

# Accessing elements of APSIM simulation

Loading APSIM simulation file

In [ ]:
with open('Maize.apsimx', 'r') as file:
    data = json.load(file)

Prints the structure at each level: can go more or less specific to find the correct level for the variable of interest.

In [9]:
data['Children'][0]['Children'][4]['Children'][7]

{'$type': 'Models.Manager, Models',
 'Code': 'using Models.Interfaces;\r\nusing System.Linq;\r\nusing System;\r\nusing Models.Core;\r\nusing Models.PMF;\r\nusing Models.Soils;\r\nusing Models.Soils.Nutrients;\r\nusing Models.Utilities;\r\nusing APSIM.Shared.Utilities;\r\nusing Models.Climate;\r\n\r\nnamespace Models\r\n{\r\n    [Serializable]\r\n    public class Script : Model\r\n    {\r\n        [Link] private Clock Clock;\r\n        [Link] private Fertiliser Fertiliser;\r\n        [Link] private Summary Summary;\r\n        [Link(ByName = true)] private Plant Maize;\r\n        [Link] private Soil Soil;\r\n        private Accumulator accumulatedRain;\r\n        [Link]\r\n        private ISoilWater waterBalance;\r\n        \r\n        [Description("Start of sowing window (d-mmm)")]\r\n        public string StartDate { get; set; }\r\n        [Description("End of sowing window (d-mmm)")]\r\n        public string EndDate { get; set; }\r\n        [Description("Minimum extractable soil water